In [1]:
#-*- coding: utf-8 -*-

In [1]:
import glob
import os
from bs4 import BeautifulSoup
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from transformers import pipeline

2025-01-30 14:52:27.605988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738245147.625718   80082 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738245147.631736   80082 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 14:52:27.652688: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def lire_fichier(chemin):
    """ Lecture de fichier"""
    fichier = open(chemin, encoding='utf-8')
    chaine = fichier.read()
    fichier.close()
    return chaine

In [71]:
# Télécharger un modèle de Question Answering
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained("timpal0l/mdeberta-v3-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

# sauvegarder en local
tokenizer.save_pretrained("models/mdeberta-v3-base-squad2")
model.save_pretrained("models/mdeberta-v3-base-squad2")

In [72]:
# ouvrir le modele 
chemin_model = "models/mdberta-v3-base-squad2/"
tokenizer = AutoTokenizer.from_pretrained(chemin_model)
model = AutoModelForQuestionAnswering.from_pretrained(chemin_model)

In [73]:
# pipeline
from transformers import pipeline
qa_model = pipeline("question-answering", model=model, tokenizer=tokenizer)

Device set to use cpu


In [74]:
import glob
from bs4 import BeautifulSoup
from tqdm import tqdm

# Étant donné qu'on a déjà travaillé sur les sections du corpus TALN-RECITAL 
# On reprend un document du même corpus
# toujours sur les sections d'un document 
# De base les documents sont en .pdf
# Pour trouver et extraire les sections d'un document on a utilisé GROBID qui est utile pour travailler sur des documents scientifiques bien structurés
# Et on extrait le contenu des sections qui se trouvent entre des balises <div> </div>

Documents = {}
a = 0
liste_fichiers = glob.glob("TALN_FR/101.pdf.tei.xml") 

for chemin in liste_fichiers:
    doc = Documents[chemin] = lire_fichier(chemin)
    soup = BeautifulSoup(doc, 'html.parser')
    toutes_sections = ""
    a = 0
    for cpt, bloc in enumerate(soup.find_all("div")):
        section = bloc.text
        if len(section) < 100:
            continue
        toutes_sections += section + "\n\n" # concaténer les sections
    print("Document : ")
    print(chemin)
    # stocker une liste de questions 
    questions = [
        "Quelles sont les missions principales de l'Institut Royal de la Culture Amazighe (IRCAM) ?",
        """Quelle est la vocation de l'application "Chiffres en Amazighe" ?""",
]
    for question in tqdm(questions, desc="Questions en cours de traitement"):
        reponse = qa_model(question=question, context=toutes_sections)
        print("Réponse : ", reponse['answer'])

Document : 
TALN_FR/101.pdf.tei.xml


Questions en cours de traitement:  50%|█████▌     | 1/2 [00:08<00:08,  8.29s/it]

Réponse :   ceux liés au support des contraintes technologiques,


Questions en cours de traitement: 100%|███████████| 2/2 [00:16<00:00,  8.06s/it]

Réponse :  l'apprentissage ludique de la langue amazighe.


In [158]:
print(liste_fichiers)

['TALN_FR/101.pdf.tei.xml', 'TALN_FR/968.pdf.tei.xml']
